In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
import dlib
import os
import pickle
np.random.seed(1000)


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
frames = []
labels = []
for file in os.listdir('output_bordam/'):
    if file[-10:] == 'frames.pkl':
        with open('output_bordam/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        with open('output_bordam/'+file, 'rb') as f:
            labels.append(pickle.load(f))

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [label[0]] * len(clip)
    except:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [label[0]] * len(clip)
    except:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [7]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [8]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [9]:
train_labels //= 2
test_labels //= 2

In [10]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=96, input_shape=(48,48,1), kernel_size=3, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=96, kernel_size=3, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=96, kernel_size=3, strides=2, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=192, kernel_size=3, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=192, kernel_size=3, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=192, kernel_size=3, strides=2, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=192, kernel_size=1, padding='valid', activation='relu'),
    keras.layers.Conv2D(filters=3, kernel_size=1, padding='valid', activation='relu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(2, activation='softmax')
])

In [11]:
model.compile(optimizer='adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
from sklearn.utils.class_weight import compute_class_weight 
weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
print(weights)

[0.68708684 1.83627784]


In [13]:
weights = {0:0.68708684, 1:1.83627784}
print(weights)

{0: 0.68708684, 1: 1.83627784}


In [14]:
model.fit(train_images, train_labels, epochs=3, class_weight=weights, batch_size=50)

  ...
    to  
  ['...']
Train on 1944475 samples
Epoch 1/3
1944475/1944475 [==============================] - 794s 409us/sample - loss: 0.6933 - accuracy: 0.6582
Epoch 2/3
1944475/1944475 [==============================] - 789s 406us/sample - loss: 0.6932 - accuracy: 0.6915
Epoch 3/3
1944475/1944475 [==============================] - 788s 405us/sample - loss: 0.6932 - accuracy: 0.7406


In [15]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

482212/482212 [==============================] - 85s 176us/sample - loss: 0.6906 - accuracy: 0.7430


In [16]:
# model.save('boredom.h5')